## Génération de données synthétiques pour la prédiction du revenu annuel

Dans cette section, nous générons un dataset synthétique de 40 000 individus pour modéliser le revenu annuel des Marocains. Les variables simulées incluent des informations démographiques (âge, sexe, milieu de résidence), éducatives, professionnelles et patrimoniales.

Les étapes principales sont :
- Création aléatoire de variables réalistes comme l'âge, le sexe, le niveau d'éducation, l'expérience, etc.
- Construction d'une variable cible `revenu_annuel` influencée par différents facteurs socio-économiques avec un ajout de bruit pour simuler la variabilité réelle.
- Introduction de valeurs manquantes (5%) dans certaines colonnes pour refléter les données incomplètes du monde réel.
- Injection de valeurs aberrantes dans le revenu pour simuler des cas extrêmes.
- Création finale d’un DataFrame Pandas, avec exportation au format CSV sous le nom `dataset_revenu_marocains.csv`.

Ce dataset servira de base à l'entraînement et à l'évaluation de modèles de machine learning pour la prédiction du revenu.


In [2]:
import pandas as pd
import numpy as np
import random

# Fixer la seed pour la reproductibilité
np.random.seed(42)

# Nombre d’enregistrements
n = 40000

# Variables de base simulées
ages = np.random.randint(18, 70, size=n)
sexes = np.random.choice(["Homme", "Femme"], size=n, p=[0.55, 0.45])
milieux = np.random.choice(["Urbain", "Rural"], size=n, p=[0.65, 0.35])
education_levels = np.random.choice(["Sans niveau", "Fondamental", "Secondaire", "Supérieur"], size=n, p=[0.25, 0.35, 0.25, 0.15])
experience = np.clip(ages - np.random.randint(16, 25, size=n), 0, None)
etat_matrimonial = np.random.choice(["Célibataire", "Marié", "Divorcé", "Veuf"], size=n, p=[0.4, 0.45, 0.1, 0.05])
possession_voiture = np.random.choice([0, 1], size=n, p=[0.7, 0.3])
possession_logement = np.random.choice([0, 1], size=n, p=[0.6, 0.4])
possession_terrain = np.random.choice([0, 1], size=n, p=[0.85, 0.15])
socio_group = np.random.choice([1, 2, 3, 4, 5, 6], size=n, p=[0.05, 0.15, 0.2, 0.25, 0.2, 0.15])

# Variables ajoutées
nbr_enfants = np.random.poisson(2, size=n)
charge_parentale = np.random.choice([0, 1], size=n, p=[0.6, 0.4])
travail_secondaire = np.random.choice([0, 1], size=n, p=[0.85, 0.15])

# Catégorie d’âge
def categoriser_age(age):
    if age < 26:
        return "Jeune"
    elif age < 40:
        return "Adulte"
    elif age < 60:
        return "Sénior"
    else:
        return "Âgé"

categorie_age = np.array([categoriser_age(a) for a in ages])

# Génération du revenu avec bruit
base_revenu = (
    12000 +
    (ages * 100) +
    (experience * 150) +
    (np.where(sexes == "Homme", 2000, -1000)) +
    (np.where(milieux == "Urbain", 6000, -3000)) +
    (np.array([{"Sans niveau": 0, "Fondamental": 2000, "Secondaire": 5000, "Supérieur": 10000}[e] for e in education_levels])) +
    (np.array([8000 - g * 1000 for g in socio_group])) +
    (possession_voiture * 3000 + possession_logement * 5000 + possession_terrain * 2000)
)

revenus = base_revenu + np.random.normal(0, 3000, size=n)
revenus = np.clip(revenus, 2000, None)  # Minimum revenu

# Injecter valeurs manquantes (5% aléatoirement)
for col in ["education_levels", "etat_matrimonial", "revenus"]:
    mask = np.random.rand(n) < 0.05
    vars()[col][mask] = None

# Créer le DataFrame
df = pd.DataFrame({
    "age": ages,
    "categorie_age": categorie_age,
    "sexe": sexes,
    "milieu": milieux,
    "niveau_education": education_levels,
    "experience": experience,
    "etat_matrimonial": etat_matrimonial,
    "possede_voiture": possession_voiture,
    "possede_logement": possession_logement,
    "possede_terrain": possession_terrain,
    "socio_pro_group": socio_group,
    "revenu_annuel": revenus,
    "nombre_enfants": nbr_enfants,
    "charge_parentale": charge_parentale,
    "travail_secondaire": travail_secondaire,
    "colonne_redundante": ages,  # Redondante
    "non_pertinente": "N/A"  # Non pertinente
})

# Injecter quelques valeurs aberrantes
df.loc[df.sample(frac=0.005).index, "revenu_annuel"] *= 5

# Sauvegarder le CSV
csv_path = "dataset_revenu_marocains.csv"
df.to_csv(csv_path, index=False)

csv_path
# Aperçu des 10 premières lignes
df.head(10)

,age,categorie_age,sexe,milieu,niveau_education,experience,etat_matrimonial,possede_voiture,possede_logement,possede_terrain,socio_pro_group,revenu_annuel,nombre_enfants,charge_parentale,travail_secondaire,colonne_redundante,non_pertinente
0,56,Sénior,Femme,Rural,Sans niveau,36,Marié,0,0,0,5,17280.125152,1,1,0,56,N/A
1,69,Âgé,Homme,Urbain,Sans niveau,53,Marié,0,1,0,4,42914.143442,1,1,0,69,N/A
2,46,Sénior,Femme,Rural,Secondaire,22,Célibataire,1,1,0,5,34253.521168,2,0,0,46,N/A
3,32,Adulte,Homme,Rural,Fondamental,15,Marié,0,0,0,3,23291.704608,2,1,1,32,N/A
4,60,Âgé,Homme,Rural,Secondaire,43,Célibataire,0,0,0,2,35530.245679,3,0,0,60,N/A
5,25,Jeune,Homme,Rural,Fondamental,4,Célibataire,0,1,0,4,28870.819392,3,0,0,25,N/A
6,38,Adulte,Homme,Urbain,Sans niveau,15,Marié,0,0,0,2,33985.541853,1,0,0,38,N/A
7,56,Sénior,Femme,Rural,Secondaire,35,Marié,0,0,1,4,27741.266025,2,1,1,56,N/A
8,36,Adulte,Homme,Rural,Sans niveau,19,Célibataire,1,0,0,3,24213.745398,2,0,0,36,N/A
9,40,Sénior,Homme,Urbain,Sans niveau,20,Marié,0,1,0,3,35895.048484,4,1,0,40,N/A


## 📋 Exploration initiale du dataset

Dans cette section, nous effectuons une première exploration du dataset généré afin de mieux comprendre sa structure et son contenu. Les actions réalisées sont les suivantes :

- 📊 Affichage du **nombre total d'observations** (lignes) et de **variables** (colonnes).
- 🔍 Inspection des **types de données** (numériques, catégorielles, booléennes, etc.) pour chaque colonne.
- 📈 Génération de **statistiques descriptives** pour les variables numériques (moyenne, écart-type, min, max, quartiles).
- 📊 Génération de statistiques pour les **variables catégorielles** (nombre de valeurs uniques, valeur la plus fréquente, fréquence associée).

Ces étapes permettent d’identifier rapidement les variables clés, les éventuelles anomalies, et de poser les bases pour les étapes de nettoyage et de visualisation.


In [2]:
# 📊 Volume et dimensions
print(f"✅ Nombre d’instances : {df.shape[0]}")
print(f"✅ Nombre de colonnes : {df.shape[1]}")

# 🔍 Types des données
print("\n📌 Types de données :")
print(df.dtypes)

# 📈 Statistiques descriptives globales
print("\n📌 Statistiques descriptives (valeurs numériques) :")
display(df.describe())

print("\n📌 Statistiques descriptives (valeurs catégorielles) :")
display(df.describe(include='object'))


✅ Nombre d’instances : 40000
✅ Nombre de colonnes : 17

📌 Types de données :
age                     int64
categorie_age          object
sexe                   object
milieu                 object
niveau_education       object
experience              int64
etat_matrimonial       object
possede_voiture         int64
possede_logement        int64
possede_terrain         int64
socio_pro_group         int64
revenu_annuel         float64
nombre_enfants          int64
charge_parentale        int64
travail_secondaire      int64
colonne_redundante      int64
non_pertinente         object
dtype: object

📌 Statistiques descriptives (valeurs numériques) :


,age,experience,possede_voiture,possede_logement,possede_terrain,socio_pro_group,revenu_annuel,nombre_enfants,charge_parentale,travail_secondaire,colonne_redundante
count,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,37993.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,43.491175,23.628675,0.300850,0.399925,0.148850,3.837275,34881.745127,2.005275,0.401550,0.150975,43.491175
std,14.976835,14.989533,0.458633,0.489889,0.355945,1.419999,12872.564645,1.420070,0.490218,0.358029,14.976835
min,18.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3975.246759,0.000000,0.000000,0.000000,18.000000
25%,31.000000,11.000000,0.000000,0.000000,0.000000,3.000000,28701.094523,1.000000,0.000000,0.000000,31.000000
50%,43.000000,23.000000,0.000000,0.000000,0.000000,4.000000,34273.345282,2.000000,0.000000,0.000000,43.000000
75%,56.000000,37.000000,1.000000,1.000000,0.000000,5.000000,39797.197853,3.000000,1.000000,0.000000,56.000000
max,69.000000,53.000000,1.000000,1.000000,1.000000,6.000000,289898.277678,9.000000,1.000000,1.000000,69.000000



📌 Statistiques descriptives (valeurs catégorielles) :


,categorie_age,sexe,milieu,niveau_education,etat_matrimonial,non_pertinente
count,40000,40000,40000,40000,40000,40000
unique,4,2,2,5,5,1
top,Sénior,Homme,Urbain,Fondamental,Marié,N/A
freq,15325,22026,25987,13223,17045,40000


In [3]:
!pip install sweetviz

## 🤖 Analyse exploratoire automatisée avec Sweetviz

Pour compléter l’exploration manuelle du dataset, nous utilisons **Sweetviz**, une bibliothèque Python qui génère automatiquement un rapport visuel interactif sur les données.

L’analyse comprend :
- Les distributions des variables numériques et catégorielles.
- La détection des valeurs manquantes.
- Les corrélations entre variables.
- Les comparaisons entre classes si une variable cible est définie (dans notre cas, ce sera utile plus tard).

Un fichier HTML nommé `rapport_complet_sweetviz.html` est généré. Il peut être ouvert dans un navigateur pour explorer facilement les caractéristiques du dataset.

> ⚠️ Assurez-vous que le fichier est bien généré dans le même répertoire que le notebook pour pouvoir le consulter sans problème.


In [4]:
# 📊 Analyse exploratoire automatisée avec Sweetviz
import sweetviz as sv

rapport = sv.analyze(df)
rapport.show_html("rapport_complet_sweetviz.html")

print("✅ Rapport Sweetviz généré : rapport_complet_sweetviz.html")


                                             |          | [  0%]   00:00 -> (? left)

Report rapport_complet_sweetviz.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
✅ Rapport Sweetviz généré : rapport_complet_sweetviz.html


## 🧼 Nettoyage des données

Avant d'entraîner un modèle de machine learning, il est essentiel de nettoyer le dataset afin d'améliorer la qualité des données et la fiabilité des résultats.

Les étapes suivantes sont effectuées :

1. **Suppression des lignes avec un revenu manquant** (`revenu_annuel`) : comme cette variable est notre cible pour la prédiction, les lignes sans cette information sont retirées.
2. **Suppression des doublons** : les entrées dupliquées sont supprimées pour éviter les biais dans l’analyse ou l'entraînement du modèle.
3. **Correction des valeurs aberrantes dans le revenu annuel** :  
   - Utilisation de la méthode de l'écart interquartile (IQR) pour détecter les revenus excessivement élevés.
   - Les valeurs au-dessus de `Q3 + 3*IQR` sont plafonnées à cette limite supérieure.

Ces opérations permettent d’assurer la **cohérence**, la **qualité** et la **pertinence statistique** des données.


In [5]:
# 🧼 6.2 – Nettoyage des données

# 1️⃣ Supprimer les lignes avec un revenu manquant
df = df[df["revenu_annuel"].notna()].copy()

# 2️⃣ Supprimer les doublons
initial_count = df.shape[0]
df.drop_duplicates(inplace=True)
print(f"✅ Doublons supprimés : {initial_count - df.shape[0]}")

# 3️⃣ Correction des valeurs aberrantes sur revenu_annuel
q1 = df["revenu_annuel"].quantile(0.25)
q3 = df["revenu_annuel"].quantile(0.75)
iqr = q3 - q1
upper_bound = q3 + 3 * iqr

aberrant_mask = df["revenu_annuel"] > upper_bound
nb_aberrants = aberrant_mask.sum()
df.loc[aberrant_mask, "revenu_annuel"] = upper_bound

print(f"✅ Valeurs aberrantes corrigées : {nb_aberrants}")


✅ Doublons supprimés : 0
✅ Valeurs aberrantes corrigées : 190


## 🔁 Transformation des données avec pipelines

Dans cette section, nous préparons les données pour l'entraînement des modèles de machine learning. Cela inclut la suppression de variables inutiles, la séparation des variables explicatives de la variable cible, ainsi que la définition d’un pipeline de transformation.

### Étapes effectuées :

1. **Suppression des colonnes inutiles** :
   - `colonne_redundante` et `non_pertinente` sont supprimées car elles n’apportent aucune information pertinente pour la modélisation.

2. **Séparation des variables** :
   - `X` contient les variables explicatives.
   - `y` contient la variable cible : `revenu_annuel`.

3. **Identification des types de variables** :
   - Les colonnes numériques et catégorielles sont automatiquement détectées pour appliquer des traitements adaptés.

4. **Construction des pipelines** :
   - 🧮 Pour les **variables numériques** : imputation des valeurs manquantes par la médiane + standardisation.
   - 🏷️ Pour les **variables catégorielles** : imputation par la modalité la plus fréquente + encodage one-hot (avec gestion des modalités inconnues).

L’ensemble est intégré dans un `ColumnTransformer` pour garantir une transformation cohérente et automatisée des données en amont de la modélisation.


In [6]:
# 🔁 6.3 – Transformation des données

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# 1️⃣ Suppression des colonnes redondantes et non pertinentes
df.drop(columns=["colonne_redundante", "non_pertinente"], inplace=True, errors="ignore")
print("✅ Colonnes redondantes et non pertinentes supprimées.")

# 2️⃣ Séparer X et y
X = df.drop("revenu_annuel", axis=1)
y = df["revenu_annuel"]

# 3️⃣ Identifier les colonnes numériques et catégorielles
numerical_cols = X.select_dtypes(include=["int64", "float64"]).columns
categorical_cols = X.select_dtypes(include=["object"]).columns

# 4️⃣ Construire le pipeline de transformation
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocessor_final = ColumnTransformer(transformers=[
    ("num", num_pipeline, numerical_cols),
    ("cat", cat_pipeline, categorical_cols)
])

print("✅ Pipeline de transformation prêt.")


✅ Colonnes redondantes et non pertinentes supprimées.
✅ Pipeline de transformation prêt.


## ✂️ Séparation des données en ensembles d'entraînement et de test

Avant d'entraîner un modèle, il est essentiel de diviser le dataset en deux sous-ensembles distincts :

- **Ensemble d'entraînement (train)** : utilisé pour ajuster les paramètres du modèle.
- **Ensemble de test (test)** : utilisé pour évaluer la performance du modèle sur des données jamais vues.

Dans notre cas :
- 70 % des données sont utilisées pour l'entraînement.
- 30 % sont réservées pour le test.
- La séparation est **aléatoire mais reproductible** grâce à l’argument `random_state=42`.

Cette séparation permet une **évaluation fiable** de la capacité du modèle à généraliser à de nouvelles données.


In [7]:
# ✂️ 6.4 – Séparation des données

from sklearn.model_selection import train_test_split

# 70% train, 30% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print(f"✅ Séparation des données effectuée :")
print(f"   ➤ Train : {X_train.shape[0]} lignes")
print(f"   ➤ Test  : {X_test.shape[0]} lignes")


✅ Séparation des données effectuée :
   ➤ Train : 26595 lignes
   ➤ Test  : 11398 lignes


## 🧠 Modélisation et recherche d’hyperparamètres

Nous testons plusieurs modèles de régression afin de prédire le **revenu annuel**. Pour chaque modèle, nous utilisons un pipeline complet incluant la transformation des données (`preprocessor_final`) et la recherche des **meilleurs hyperparamètres** à l’aide de `RandomizedSearchCV`.

### Modèles évalués :
- 🔹 `LinearRegression` : modèle de base, sans hyperparamètres.
- 🌲 `DecisionTreeRegressor` : arbre de décision avec recherche sur la profondeur, le critère, etc.
- 🌳 `RandomForestRegressor` : forêt aléatoire avec ajustement du nombre d’arbres et de leur profondeur.
- 🔺 `GradientBoostingRegressor` : boosting par gradient avec tuning du taux d’apprentissage, du nombre d’arbres, etc.
- 🧠 `MLPRegressor` : réseau de neurones avec arrêt anticipé et ajustement de plusieurs paramètres (couches, activation, taux d’apprentissage...).

### Méthodologie :
- Utilisation de `RandomizedSearchCV` avec 3 **folds** pour validation croisée.
- **Score d’optimisation** : `R²` sur les données de validation.
- **Évaluation finale** sur l’ensemble de test à l’aide des métriques :
  - R² (coefficient de détermination)
  - MAE (erreur absolue moyenne)
  - RMSE (racine de l’erreur quadratique moyenne)

À la fin, un **résumé comparatif** présente les performances et les meilleurs paramètres pour chaque modèle.

> ⚙️ N_iter est fixé à 5 pour accélérer les recherches, mais il peut être augmenté pour améliorer les résultats.


In [12]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline
import numpy as np
import pandas as pd

# Modèles et hyperparamètres
models = {
    "LinearRegression": {
        "estimator": make_pipeline(preprocessor_final, LinearRegression()),
        "params": {}
    },
    "DecisionTreeRegressor": {
        "estimator": make_pipeline(preprocessor_final, DecisionTreeRegressor(random_state=42)),
        "params": {
            "decisiontreeregressor__criterion": ["squared_error", "absolute_error"],
            "decisiontreeregressor__max_depth": [None, 5, 6, 7, 10],
            "decisiontreeregressor__min_samples_split": [2, 3, 4, 5, 10]
        }
    },
    "RandomForestRegressor": {
        "estimator": make_pipeline(preprocessor_final, RandomForestRegressor(random_state=42)),
        "params": {
            "randomforestregressor__n_estimators": [50, 100, 150, 200],
            "randomforestregressor__criterion": ["squared_error", "absolute_error"],
            "randomforestregressor__max_depth": [None, 5, 10, 15, 20]
        }
    },
    "GradientBoostingRegressor": {
        "estimator": make_pipeline(preprocessor_final, GradientBoostingRegressor(random_state=42)),
        "params": {
            "gradientboostingregressor__loss": ["squared_error", "absolute_error"],
            "gradientboostingregressor__learning_rate": [0.01, 0.1, 0.2],
            "gradientboostingregressor__n_estimators": [100, 200, 300],
            "gradientboostingregressor__subsample": [0.5, 0.8, 1]
        }
    },
    "MLPRegressor": {
    "estimator": make_pipeline(
        preprocessor_final,
        MLPRegressor(
            random_state=42,
            early_stopping=True,     # ✅ stoppe si pas d’amélioration
            tol=1e-4,                # ✅ tolérance plus stricte
            max_fun=15000            # ✅ empêche trop d’appels internes
        )
    ),
    "params": {
        "mlpregressor__hidden_layer_sizes": [(50,), (100,), (100, 50), (100, 100)],
        "mlpregressor__activation": ["relu", "tanh", "logistic"],
        "mlpregressor__solver": ["adam", "sgd"],
        "mlpregressor__alpha": [0.0001, 0.001, 0.01],
        "mlpregressor__learning_rate": ["constant", "invscaling", "adaptive"],
        "mlpregressor__learning_rate_init": [0.001, 0.01, 0.1],
        "mlpregressor__max_iter": [100, 200, 300]
    }
}
}

# Résultats
results = []

for name, config in models.items():
    print(f"🔄 Entraînement (RandomizedSearchCV) pour : {name}")
    search = RandomizedSearchCV(
        estimator=config["estimator"],
        param_distributions=config["params"],
        n_iter=5,  # 🟡 Tu peux ajuster à 5 ou 3 si c’est encore lent
        scoring="r2",
        cv=3,
        n_jobs=-1,
        random_state=42
    )
    
    search.fit(X_train, y_train)
    y_pred = search.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)
    print("📊 Résultats :")
    print(f"   ➤ R2 (test)  : {r2:.4f}")
    print(f"   ➤ MAE (test) : {mae:.2f}")
    print(f"   ➤ RMSE (test): {rmse:.2f}")
    print(f"   ➤ Meilleurs hyperparamètres : {search.best_params_}")
    results.append({
        "Modèle": name,
        "Meilleurs paramètres": search.best_params_,
        "R2 (validation)": search.best_score_,
        "MAE (test)": mae,
        "RMSE (test)": rmse,
        "R2 (test)": r2,
        "Pipeline": search.best_estimator_
    })

# Résumé des résultats
results_df = pd.DataFrame(results).sort_values(by="R2 (test)", ascending=False)
results_df[["Modèle", "R2 (test)", "MAE (test)", "RMSE (test)", "Meilleurs paramètres"]]


🔄 Entraînement (RandomizedSearchCV) pour : LinearRegression


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 1 is smaller than n_iter=5. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


📊 Résultats :
   ➤ R2 (test)  : 0.7663
   ➤ MAE (test) : 2608.12
   ➤ RMSE (test): 4069.83
   ➤ Meilleurs hyperparamètres : {}
🔄 Entraînement (RandomizedSearchCV) pour : DecisionTreeRegressor
📊 Résultats :
   ➤ R2 (test)  : 0.6982
   ➤ MAE (test) : 3070.21
   ➤ RMSE (test): 4624.90
   ➤ Meilleurs hyperparamètres : {'decisiontreeregressor__min_samples_split': 2, 'decisiontreeregressor__max_depth': 10, 'decisiontreeregressor__criterion': 'absolute_error'}
🔄 Entraînement (RandomizedSearchCV) pour : RandomForestRegressor
📊 Résultats :
   ➤ R2 (test)  : 0.7203
   ➤ MAE (test) : 2921.76
   ➤ RMSE (test): 4452.03
   ➤ Meilleurs hyperparamètres : {'randomforestregressor__n_estimators': 200, 'randomforestregressor__max_depth': 15, 'randomforestregressor__criterion': 'squared_error'}
🔄 Entraînement (RandomizedSearchCV) pour : GradientBoostingRegressor
📊 Résultats :
   ➤ R2 (test)  : 0.7623
   ➤ MAE (test) : 2645.78
   ➤ RMSE (test): 4104.41
   ➤ Meilleurs hyperparamètres : {'gradientboostingregr

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptro

📊 Résultats :
   ➤ R2 (test)  : 0.7331
   ➤ MAE (test) : 2879.04
   ➤ RMSE (test): 4348.77
   ➤ Meilleurs hyperparamètres : {'mlpregressor__solver': 'adam', 'mlpregressor__max_iter': 200, 'mlpregressor__learning_rate_init': 0.1, 'mlpregressor__learning_rate': 'adaptive', 'mlpregressor__hidden_layer_sizes': (50,), 'mlpregressor__alpha': 0.01, 'mlpregressor__activation': 'tanh'}


,Modèle,R2 (test),MAE (test),RMSE (test),Meilleurs paramètres
0,LinearRegression,0.766278,2608.115165,4069.831167,{}
3,GradientBoostingRegressor,0.762289,2645.779093,4104.411564,"{'gradientboostingregressor__subsample': 0.8, ..."
4,MLPRegressor,0.733142,2879.037074,4348.774920,"{'mlpregressor__solver': 'adam', 'mlpregressor..."
2,RandomForestRegressor,0.720319,2921.762837,4452.028848,"{'randomforestregressor__n_estimators': 200, '..."
1,DecisionTreeRegressor,0.698177,3070.214191,4624.904614,{'decisiontreeregressor__min_samples_split': 2...


## ✅ Test final du meilleur modèle

Après l’évaluation de plusieurs modèles, nous sélectionnons **celui qui présente le meilleur score R²** sur les données de test.

### Étapes réalisées :
1. **Identification automatique** du modèle le plus performant (`best_model`) selon le R².
2. **Prédiction** sur l’ensemble de test (`X_test`) avec ce modèle.
3. **Évaluation finale** à l’aide des métriques :
   - MAE (Erreur absolue moyenne)
   - RMSE (Racine de l’erreur quadratique moyenne)
   - R² (Coefficient de détermination)

> 🎯 Ce test final permet de valider que le modèle sélectionné est capable de **généraliser correctement** à de nouvelles données.


In [13]:
# 6.6 – Test final du meilleur modèle

# 1️⃣ Identifier le modèle le plus performant (meilleur R²)
best_model_info = results_df.sort_values(by="R2 (test)", ascending=False).iloc[0]
best_model_name = best_model_info["Modèle"]
best_model = best_model_info["Pipeline"]

print(f"\n✅ Test du modèle sélectionné : {best_model_name}")

# 2️⃣ Prédiction sur X_test
y_pred = best_model.predict(X_test)

# 3️⃣ Évaluation finale
mae_final = mean_absolute_error(y_test, y_pred)
rmse_final = mean_squared_error(y_test, y_pred, squared=False)
r2_final = r2_score(y_test, y_pred)

print("📊 Résultats finaux du modèle sélectionné :")
print(f"   ➤ MAE  : {mae_final:.2f}")
print(f"   ➤ RMSE : {rmse_final:.2f}")
print(f"   ➤ R²   : {r2_final:.4f}")



✅ Test du modèle sélectionné : LinearRegression
📊 Résultats finaux du modèle sélectionné :
   ➤ MAE  : 2608.12
   ➤ RMSE : 4069.83
   ➤ R²   : 0.7663


## 💾 Sauvegarde des meilleurs modèles

Afin de pouvoir **réutiliser les modèles sans devoir les réentraîner**, nous sauvegardons les **3 meilleurs modèles** (selon leur score R² sur l’ensemble de test) au format `.joblib`.

### Pourquoi sauvegarder les modèles ?
- Pour une utilisation dans une **application web** ou une **API de prédiction**.
- Pour **partager** le modèle avec d'autres utilisateurs.
- Pour **gagner du temps** lors de futures prédictions.

> 📂 Les fichiers générés peuvent ensuite être rechargés avec `joblib.load("nom_du_fichier.joblib")`.


In [14]:
import joblib

# Sauvegarder les 3 meilleurs modèles dans des fichiers .joblib
top_3 = results_df.sort_values(by="R2 (test)", ascending=False).head(3)

saved_models = []
for i, row in top_3.iterrows():
    model_name = row["Modèle"].lower().replace(" ", "_")
    filename = f"{model_name}_model.joblib"
    joblib.dump(row["Pipeline"], filename)
    saved_models.append((row["Modèle"], filename))

saved_models


[('LinearRegression', 'linearregression_model.joblib'),
 ('GradientBoostingRegressor', 'gradientboostingregressor_model.joblib'),
 ('MLPRegressor', 'mlpregressor_model.joblib')]